# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file created in Part I
city_data_csv = "../WeatherPy/output_data/cities.csv"
# Create dataframe
city_data_df = pd.read_csv(city_data_csv)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,waimana,-38.1500,177.0833,66.00,55.0,14.0,5.99,NZ,1.611958e+09
1,tikrit,34.6071,43.6782,49.55,76.0,34.0,6.26,IQ,1.611958e+09
2,isangel,-19.5500,169.2667,82.40,74.0,20.0,15.88,VU,1.611958e+09
3,norman wells,65.2820,-126.8329,-31.00,75.0,20.0,3.44,CA,1.611958e+09
4,rikitea,-23.1203,-134.9692,80.11,67.0,88.0,7.54,PF,1.611958e+09
...,...,...,...,...,...,...,...,...,...
573,egvekinot,66.3167,-179.1667,-21.78,84.0,2.0,10.40,RU,1.611958e+09
574,altay,47.8667,88.1167,3.42,78.0,48.0,1.83,CN,1.611958e+09
575,katsuura,35.1333,140.3000,36.00,48.0,20.0,19.57,JP,1.611958e+09
576,sao paulo de olivenca,-3.3783,-68.8725,79.56,92.0,100.0,1.12,BR,1.611958e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
# Convert Humidity to float and store
# HINT: be sure to handle NaN values
city_data_df = city_data_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Create a Humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,saint-pierre,-21.3393,55.4781,78.80,65.0,0.0,6.91,RE,1.611958e+09
215,wichian buri,15.6578,101.1060,70.52,61.0,0.0,8.21,TH,1.611958e+09
216,saint-philippe,-21.3585,55.7679,78.80,65.0,0.0,6.91,RE,1.611958e+09
269,mattru,7.6244,-11.8332,77.41,84.0,0.0,3.83,SL,1.611958e+09
288,uruguaiana,-29.7547,-57.0883,73.40,88.0,0.0,2.30,BR,1.611958e+09
293,les cayes,18.2000,-73.7500,79.12,65.0,0.0,5.44,HT,1.611958e+09
471,manjacaze,-24.7117,33.8828,73.69,96.0,0.0,2.01,MZ,1.611958e+09
537,grand gaube,-20.0064,57.6608,77.00,82.0,0.0,3.00,MU,1.611958e+09
541,caravelas,-17.7125,-39.2481,77.45,80.0,0.0,6.91,BR,1.611958e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
127,saint-pierre,RE,-21.3393,55.4781,
215,wichian buri,TH,15.6578,101.1060,
216,saint-philippe,RE,-21.3585,55.7679,
269,mattru,SL,7.6244,-11.8332,
288,uruguaiana,BR,-29.7547,-57.0883,
293,les cayes,HT,18.2000,-73.7500,
471,manjacaze,MZ,-24.7117,33.8828,
537,grand gaube,MU,-20.0064,57.6608,
541,caravelas,BR,-17.7125,-39.2481,


In [11]:
# Set parameters to serach for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while keeping original params in place
    params["location"] = f"{lat},{lng}"
    
    # Create base url to search nearby hotel     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        print(f"Closest hotel is {hotel_name[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display hotel names on dataframe
hotel_df

Retrieving results for city: saint-pierre
Closest hotel is Lindsey Hôtel.
------------
Retrieving results for city: wichian buri
Closest hotel is Mit Sali Hotel.
------------
Retrieving results for city: saint-philippe
Closest hotel is Chambres d'hôte "La Trinité".
------------
Retrieving results for city: mattru
Closest hotel is Mondkor Town, Bagbo Chiefdom, Bo District.
------------
Retrieving results for city: uruguaiana
Closest hotel is Fares Turis Hotel.
------------
Retrieving results for city: les cayes
Closest hotel is Villa Orphee.
------------
Retrieving results for city: manjacaze
Closest hotel is Hotel Gani.
------------
Retrieving results for city: grand gaube
Closest hotel is Veranda Paul et Virginie Hotel & Spa.
------------
Retrieving results for city: caravelas
Closest hotel is Pousada dos Navegantes.
------------


,City,Country,Lat,Lng,Hotel Name
127,saint-pierre,RE,-21.3393,55.4781,Lindsey Hôtel
215,wichian buri,TH,15.6578,101.1060,Mit Sali Hotel
216,saint-philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
269,mattru,SL,7.6244,-11.8332,"Mondkor Town, Bagbo Chiefdom, Bo District"
288,uruguaiana,BR,-29.7547,-57.0883,Fares Turis Hotel
293,les cayes,HT,18.2000,-73.7500,Villa Orphee
471,manjacaze,MZ,-24.7117,33.8828,Hotel Gani
537,grand gaube,MU,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
541,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))